#**SCRAPING PHASE**

In [ ]:
from bs4 import BeautifulSoup 
import requests
import string
import json
import re

In [ ]:
# Build up the link
inizio_link = 'https://vie.openalfa.it/'

quartieri_zona = ['brera', 'duomo', 'giardini-porta-venezia', 'guastalla_municipio-1', 'magenta-san-vittore', 'pagano', 'parco-sempione',
                'adriano', 'Centrale', 'Greco', 'Loreto_municipio-2', 'Padova_municipio-2', 'Viale-Monza', 'Buenos-Aires-Venezia', 'Citta-Studi',
                'Lambrate', 'Parco-Forlanini-Ortica', 'Parco-Lambro-Cimiano', 'Corsica', 'Lodi-Corvetto', 'Mecenate', 'Ortomercato',
                'Parco-Monlue-Ponte-Lambro' ,'Porta-Romana_municipio-4', 'Rogoredo', 'Triulzo-Superiore', 'Umbria-Molise', 'XXII-Marzo',
                'Cantalupa_milano', 'Chiaravalle_municipio-5', 'Chiesa-Rossa', 'Ex-Om-Morivione', 'Gratosoglio-Ticinello', 'Morivione', 'Parco-delle-Abbazie',
                'Quintosole', 'Ripamonti', 'Scalo-Romana', 'Stadera', 'Tibaldi', 'Ticinese', 'Vigentina', 'Bande-Nere', 'Barona', 'Boffalora_milano',
                'Cantalupa_municipio-6', 'Lorenteggio', 'Moncucco', 'Navigli', 'Parco-dei-Navigli', 'Ronchetto-sul-Naviglio', 'San-Cristoforo_milano', 'Tortona_municipio-6',
                'Baggio', 'De-Angeli-Monte-Rosa', 'Figino', 'Forze-Armate', 'Muggiano', 'Parco-Agricolo-Sud', 'Parco-Bosco-In-Citta', 'Quarto-Cagnino',
                'Quinto-Romano', 'San-Siro_municipio-7', 'Selinunte', 'Washington', 'Bullona', 'Cagnola_municipio-8-di-milano', 'Cascina-Triulza-Expo', 'Cassina-Triulza', 'Gallaratese_municipio-8',
                'Ghisolfa', 'Lampugnano', 'Maggiore-Musocco', 'Portello', 'Qt-8', 'Quarto-Oggiaro', 'Roserio', 'Sacco_municipio-8', 'Sarpi', 'Stephenson', 'Tre-Torri_municipio-8',
                'Trenno', 'Villapizzone', 'Affori', 'Bicocca', 'Bovisa', 'Bovisasca', 'Bruzzano', 'Comasina', 'Dergano', 'farini_municipio-9', 'GARIBALDI-REPUBBLICA',
                'Isola', 'Maciachini-Maggiolina', 'Niguarda-Ca-Granda', 'Parco-Nord']

quartieri_zona = [quartiere.lower() for quartiere in quartieri_zona]
pagine = ['','?pg=2', '?pg=3', '?pg=4']


def get_data(url): # this function takes the link and get the data of the page of the site
  req = requests.get(url)
  soup = BeautifulSoup(req.text, "html.parser")
  return soup 

def aggiusta_pagine(pagina): # The data are temporarely stored in a dictionary. This def fix an issue with the keys of this dictionary
  if pagina == '':
    return '1'
  else:
    return pagina.replace("?pg=", "")


def elabora_dati(zuppa): # this function processes the data obtained through get-method and filter only the streets from the html page
  vie_raw = {}
  vie_clean = {}
  count = 0
  if zuppa.find_all('div', class_='street-columns') != []:
    for item in zuppa.find_all('div', class_='street-columns')[0].find_all('li'):
      if item.find('h4') is None:
        vie_raw[count] = item.get_text().strip()
        count += 1  
      vie_clean = {chiave: re.sub(r'\([^)]*\)', '', valore.lower()) for chiave, valore in vie_raw.items()} 
  return vie_clean


diz_quartieri = {} # here the dictionary is created. It will hase the districs as keys, inside it the pages with the streets
for quartiere in quartieri_zona:
  diz_pagine = {}
  for pagina in pagine:
    url = f"""https://vie.openalfa.it/{quartiere}{pagina}"""
    zuppa = get_data(url)
    diz_pagine[aggiusta_pagine(pagina)] = elabora_dati(zuppa) 
  diz_quartieri[quartiere] = diz_pagine 


In [ ]:
# The data are stored into a json. 
with open('/content/drive/MyDrive/Progetto Data Management/Data/quartieri_raw.json', 'w') as file:
  json.dump(diz_quartieri, file)

#**DATA CLEANING PHASE**

###Null Data Removal

In [ ]:
for i in set(diz_quartieri.keys()):
  for j in set(diz_quartieri[i].keys()):
    if diz_quartieri[i][j] == {}:
      del diz_quartieri[i][j]

###*Fit for use*: we remove the pages inside the dictionary to obtain a new dictionary with districts as keys and lists of streets for values

In [ ]:
diz_senza_pagine  = {}
for i in set(diz_quartieri.keys()):
  valori = []
  for j in set(diz_quartieri[i].keys()):
    for k in set(diz_quartieri[i][j].keys()):
      valori.append(diz_quartieri[i][j][k])
  diz_senza_pagine[i] = valori

###Syntax cleaning
 

In [ ]:
# Removes punctuation
# Removes accents by converting letters with accent to their unaccented version
from unidecode import unidecode

for chiave in diz_senza_pagine.keys():
  for i in range(len(diz_senza_pagine[chiave])):
    diz_senza_pagine[chiave][i]=re.sub(r"[^\w\s]", "", diz_senza_pagine[chiave][i])
    diz_senza_pagine[chiave][i] = unidecode(diz_senza_pagine[chiave][i])

###Checking for Duplicates

In [ ]:
# Removal of duplicates for each district
diz_quartieri_clean1 = {}
for quartiere, vie in diz_senza_pagine.items():
  diz_quartieri_clean1[quartiere] = list(set(vie))

# Unfortunately, the boundary streets between one district and another belong to both. We eliminate these kind of duplicates
diz_quartieri_clean2 = diz_quartieri_clean1
for i in diz_quartieri_clean1.keys():
  for j in diz_quartieri_clean1.keys():
    if i != j:
      diz_quartieri_clean2[i] = [via for via in diz_quartieri_clean2[i] if via not in diz_quartieri_clean1[j]]

#**DATA STORAGE PHASE**

In [ ]:
#The FINAL data are stored into a json, ready for data integration
with open('/content/drive/MyDrive/Progetto Data Management/Data/quartieri.json', 'w') as file:
  json.dump(diz_quartieri_clean2, file)